In [108]:
import requests
import pandas as pd


base_url = "https://data.smartdublin.ie/sonitus-api"
realtime_url = "/api/monitors"
api_url = base_url+realtime_url

query = {"username": "dublincityapi", 
"password": "Xpa5vAQ9ki",
"monitor": "10.1.1.1",
"start":"1681309601",
"end":"1681396001"}

response = requests.post(api_url,json=query)

data = response.json()
for loc in data:
    try:
        print(float(loc['latitude']))
    except ValueError:
        print("could not convert",loc['latitude'],"to float")


53.369864
53.36866
53.343337
53.390401
53.346116
53.319492
53.323524
53.370758
53.379996
53.340031
53.346694
53.357153
53.331059
53.323204
53.390338
53.357153
53.3279804
53.346665
could not convert  to float
53.369847
53.3442389
53.3680667
53.3644417
53.390281
53.340148
53.3362889
53.3856444
53.2857556
53.2805333
53.345707
53.318902
53.369894
53.390433
53.347863
53.390542
53.352988
53.351671
53.341938
53.322029
53.372904
53.345053
53.3699
53.3687022
53.33192
53.357689
53.343628
53.318902
53.369894
53.322562
53.390433
53.356816
53.333691
53.337909


In [109]:
data = [x for x in data if x['latitude']!=""]
data=  [x for x in data if ('Noise' in x['label']!="")]

In [110]:
from arcgis.gis import GIS
from arcgis.geometry import Point
from arcgis.features import FeatureLayer, FeatureSet
from arcgis.widgets import MapView

gis = GIS(api_key = "AAPKa873723e22cb4fbea914d35e64d85b84B4qitV1d04kTbB_ZH7VcVDaG46maiKVNzixqfPpMUkZQjN58IkvTtoc9Un0_HPeR")
map1 = gis.map("Dublin", zoomlevel=12)


In [111]:
# Define the coordinates of the locations to add to the map
sp_ref = "wkid"
sp_ref_no = 4326


{'wkid': 4326}

In [112]:
#API query 
base_url = "https://data.smartdublin.ie/sonitus-api"
hourly_averages = "/api/hourly-averages"
api_url = base_url+hourly_averages
            
# Create a FeatureSet with the location data
features = []
for loc in data: 
    query = {"username": "dublincityapi", 
    "password": "Xpa5vAQ9ki",
    "monitor": loc['serial_number'],
    "start":"1641081600",
    "end":"1641081600"}
    response = requests.post(api_url, query)
    r = response.json()
    if len(r) > 0:
        try:
            point = Point({"x":float(loc['longitude']), "y": float(loc['latitude']), "spatialReference":{sp_ref:sp_ref_no}})
        except ValueError:
            point = Point({"x":loc['longitude'], "y": loc['latitude'], "spatialReference":{sp_ref:sp_ref_no}})
        feature = {"attributes": {"Name": loc["location"],  "serialnumber":loc['serial_number'], "Laeq": float(r[0]['laeq'])}, "geometry": point}
        features.append(feature)
        
feature_set = FeatureSet.from_dict({"features": features, "geometryType": "esriGeometryPoint", "spatialReference": {sp_ref:sp_ref_no}})

# Create a FeatureLayer from the FeatureSet
layer = FeatureLayer(feature_set, gis=gis)

# Print the URL of the new Feature Layer
layer.url


<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
{"features": [{"geometry": {"x": -6.258966, "y": 53.369864, "spatialReference": {"wkid": 4326}}, "attributes": {"Name": "Drumcondra Library", "serialnumber": "10.1.1.1", "Laeq": 50.0, "OBJECTID": 1}}, {"geometry": {"x": -6.149316, "y": 53.36866, "spatialReference": {"wkid": 4326}}, "attributes": {"Name": "Bull Island", "serialnumber": "01749", "Laeq": 57.0, "OBJECTID": 2}}, {"geometry": {"x": -6.264755, "y": 53.390401, "spatialReference": {"wkid": 4326}}, "attributes": {"Name": "Ballymun", "serialnumber": "10118", "Laeq": 59.4, "OBJECTID": 3}}, {"geometry": {"x": -6.321013, "y": 53.346116, "spatialReference": {"wkid": 4326}}, "attributes": {"Name": "DCC Rowing Club", "serialnumber": "01548", "Laeq": 52.4, "OBJECTID": 4}}, {"geometry": {"x": -6.321945, "y": 53.319492, "spatialReference"

In [114]:
""" map1.add_layer(layer.url, {"renderer":"HeatmapRenderer", "field_name": "Laeq", "opacity":0.95 })"""

' map1.add_layer(layer.url, {"renderer":"HeatmapRenderer", "field_name": "Laeq", "opacity":0.95 })\nmap1 '

In [115]:
import matplotlib.pyplot as plt
from arcgis.features import GeoAccessor, GeoSeriesAccessor
sdf = feature_set.sdf
sdf.spatial.plot(map_widget=map1, renderer_type = "p")

True

In [116]:
map1

MapView(layout=Layout(height='400px', width='100%'))